# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from typing import Union
import random
from pathlib import Path

In [2]:
from Return_dataloader import pass_dataloader
from Transformer_model import TabTransformer
from Plot_Accuracies import plot_loss_curves
from engine import train

## Device Agnostic

In [3]:
# Clear Cache
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Available device is: {device}")

Available device is: cpu


## Dataloader

In [4]:
train_dataloader, test_dataloader = pass_dataloader()

# Hyperparameter Training

In [5]:
from copy import deepcopy

def hyperparameter_tuning(param_grid,
                          train_dataloader,
                          test_dataloader,
                          device,
                          epochs=50):

    best_val_score = -float('inf')
    best_params = None
    best_results = None

    for params in param_grid:
        print(f"Training with d_model={params['d_model']}, nhead={params['nhead']}, "
              f"num_layers={params['num_layers']}, dropout={params['dropout']}, lr={params['lr']}")

        # Create model with given hyperparameters
        model = TabTransformer(input_dim=100,
                               d_model=params['d_model'],
                               nhead=params['nhead'],
                               num_layers=params['num_layers'],
                               dropout=params['dropout']).to(device)

        loss_fn = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=params['lr'],
                                     weight_decay=1e-4)

        # Train model and get results dictionary
        results = train(model=model,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        device=device,
                        epochs=epochs)

        # Pick best validation score (ROC-AUC here) from all epochs
        max_val_roc_auc = max(results['test_roc_auc'])

        print(f"Max val ROC-AUC: {max_val_roc_auc:.4f}")

        if max_val_roc_auc > best_val_score:
            best_val_score = max_val_roc_auc
            best_params = params
            best_results = deepcopy(results)

    print(f"\nBest params: {best_params}")
    print(f"Best validation ROC-AUC: {best_val_score:.4f}")

    return best_params, best_results


In [6]:
from itertools import product
NUM_EPOCHS = 10


# param_grid = {
#     "d_model": [64, 128, 25],
#     "nhead": [2, 4, 8, 16],
#     "num_layers": [i * 2 for i in range(1,51)],
#     "dropout": [0.0, 0.1 , 0.3 , 0.5],
#     "lr": [0.1, 0.01, 0.001],
#     "weight_decay": [0, 1e-4],
#     "batch_size": [4, 8, 16, 32, 64]
# }
param_grid = {
    "d_model": [64, 128],
    "nhead": [2, 4],
    "num_layers": [2, 4, 6],
    "dropout": [0.1, 0.3],
    "lr": [0.001, 0.0001],
    "weight_decay": [0, 1e-4],
    "batch_size": [16, 32]
}


param_grid = [
    dict(zip(param_grid.keys(), values))
    for values in product(*param_grid.values())
]
best_params, best_results = hyperparameter_tuning(
    param_grid,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    device=device,
    epochs=NUM_EPOCHS
)

Training with d_model=64, nhead=2, num_layers=2, dropout=0.1, lr=0.001


G:\M.Sc_Neaz\08_TCGA_Prostate_Data\Prostate_Cancer_Challenge\Python Scripts\pcvenv\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.0311 | Acc: 0.9899 | Prec: 0.9898 | Recall: 0.9884 | F1: 0.9891 | ROC-AUC: 0.9993 || Test Loss: 0.3068 | Acc: 0.9349 | Prec: 0.9769 | Recall: 0.9524 | F1: 0.9645 | ROC-AUC: 0.9320
Max val ROC-AUC: 0.9563
Training with d_model=64, nhead=2, num_layers=2, dropout=0.1, lr=0.001


  0%|          | 0/10 [00:00<?, ?it/s]

...

Epoch: 10 | Train Loss: 0.0510 | Acc: 0.9848 | Prec: 0.9904 | Recall: 0.9768 | F1: 0.9835 | ROC-AUC: 0.9977 || Test Loss: 0.3717 | Acc: 0.9163 | Prec: 0.9840 | Recall: 0.9248 | F1: 0.9535 | ROC-AUC: 0.9365
Max val ROC-AUC: 0.9698
Training with d_model=128, nhead=4, num_layers=6, dropout=0.3, lr=0.0001


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.0571 | Acc: 0.9848 | Prec: 0.9868 | Recall: 0.9804 | F1: 0.9836 | ROC-AUC: 0.9958 || Test Loss: 0.3753 | Acc: 0.9186 | Prec: 0.9840 | Recall: 0.9273 | F1: 0.9548 | ROC-AUC: 0.9210
Max val ROC-AUC: 0.9577
Training with d_model=128, nhead=4, num_layers=6, dropout=0.3, lr=0.0001


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 10 | Train Loss: 0.0337 | Acc: 0.9906 | Prec: 0.9941 | Recall: 0.9855 | F1: 0.9898 | ROC-AUC: 0.9985 || Test Loss: 0.3949 | Acc: 0.9186 | Prec: 0.9866 | Recall: 0.9248 | F1: 0.9547 | ROC-AUC: 0.9394
Max val ROC-AUC: 0.9550

Best params: {'d_model': 128, 'nhead': 4, 'num_layers': 4, 'dropout': 0.1, 'lr': 0.0001, 'weight_decay': 0, 'batch_size': 16}
Best validation ROC-AUC: 0.9723


### Best params: 
Best params: {'d_model': 128, 'nhead': 4, 'num_layers': 4, 'dropout': 0.1, 'lr': 0.0001, 'weight_decay': 0, 'batch_size': 16}
Best validation ROC-AUC: 0.9723